In [ ]:
#this is simple example showing the difference between performance using pandas which runs on CPU and cudf which runs on GPU
import pandas as pd
import numpy as np
import cudf
import dask_cudf
pandas_df = pd.DataFrame({'a': np.random.randint(0, 100000000, size=1000000000),
                          'b': np.random.randint(0, 100000000, size=1000000000)})
                          
cudf_df = cudf.DataFrame.from_pandas(pandas_df)
dask_df = dask_cudf.from_cudf(cudf_df,npartitions=2)
# Timing Pandas
%timeit pandas_df.a.mean()
# Timing cuDF

%timeit cudf_df.a.mean()
# timing dask_cudf
%timeit dask_df.a.mean().compute()


In [2]:
#reading the files using pandas and cudf
df=pd.read_csv('Data_outcome_deidentified.csv')
cdf= cudf.read_csv('Data_outcome_deidentified.csv')

In [3]:
#dropping unecessary columns
df.drop(['POST','PRE','causeofdeath','primcauseofdeath','death.info','numdisvessels','maxlesionlength', 'wposttimi', 'numbms','numdes', 'numstents', 'maxdevdiameter', 'maxdevlength','otherventsupport\nYes:1 \nNo:0'],axis=1,inplace=True)
cdf.drop(['POST','PRE','causeofdeath','primcauseofdeath','death.info','numdisvessels','maxlesionlength', 'wposttimi', 'numbms','numdes', 'numstents', 'maxdevdiameter', 'maxdevlength','otherventsupport\nYes:1 \nNo:0'],axis=1,inplace=True)

In [4]:
#consists of all the effects i.e. targets
effect = ['Inhospital mortality\nYes:1\nNo:0','Mortality 6 months\nYes:1\nNo:0', 'Mortality 1 yr\nYes:1\nNo:0','Mortality 2 yr\nYes:1\nNo:0', 'Mortality 5yr\nYes:1\nNo:0','Time from PCI to Stroke_6mo', 'Time from PCI to Stroke_1yr','Time from PCI to Stroke_2yr', 'Time from PCI to Stroke_5yr']

In [5]:
df_cause = cdf.drop(effect,axis=1)
cdf_cause = cdf.drop(effect,axis=1)

In [7]:
#this is using cudf the functions and process is same pandas 
%%time
from datetime import datetime
s=datetime.now()
def effect_prob():
    names=[]
    for e in effect:
        rel_c1=[]
    
        cdf_e1 = cdf[cdf[e]>-1]
        cdf_e1_prob = cdf_e1[e].sum()/len(cdf_e1[e])
        print(e,cdf_e1_prob)
        for i in cdf_cause.columns:
            cdf_e1_c1_prob=0
            cdf_e1_c1 = cdf_e1[cdf_e1[i]>-1]
            if(cdf_e1_c1[i].sum()!=0):
                #print(i,e)
                cdf_e1_c1_prob = (cdf_e1_c1[e].mul(cdf_e1_c1[i])).sum()/cdf_e1_c1[i].sum()
            if(cdf_e1_c1_prob>cdf_e1_prob):
                rel_c1.append(i)
        #print(rel_c1)
        diff=[0 for k in rel_c1]
        for i in rel_c1:
            ind=rel_c1.index(i)
            for j in rel_c1:
                if(i is not j):
                    cdf_e1_c1 = cdf_e1[cdf_e1[i]>-1]
                    cdf_e1_c1_c2 = cdf_e1_c1[cdf_e1_c1[j]>-1]
                    if(((cdf_e1_c1_c2[i].mul(cdf_e1_c1_c2[j])).sum())!=0 and ((1-cdf_e1_c1_c2[i]).mul(cdf_e1_c1_c2[j])).sum()!=0):
                        cdf_e1_c1_c2_prob = (cdf_e1_c1_c2[i].mul(cdf_e1_c1_c2[j]).mul(cdf_e1_c1_c2[e])).sum()/(cdf_e1_c1_c2[i].mul(cdf_e1_c1_c2[j])).sum()

                        cdf_e1_notc1_c2_prob = ((1-cdf_e1_c1_c2[i]).mul(cdf_e1_c1_c2[j]).mul(cdf_e1_c1_c2[e])).sum()/((1-cdf_e1_c1_c2[i]).mul(cdf_e1_c1_c2[j])).sum()

                        e_diff=cdf_e1_c1_c2_prob-cdf_e1_notc1_c2_prob

                        diff[ind]=diff[ind]+e_diff
            diff[rel_c1.index(i)] = diff[rel_c1.index(i)]/(len(rel_c1)-1) 
        print(diff)
        names.append(rel_c1)
    return names

effect_prob()
print(datetime.now()-s)
#print(rel_c1_d)

Inhospital mortality
Yes:1
No:0 0.024114366506951534
[0.35741961909329045, 0.048567734891089315, 0.00469689021099746, 0.025634407106949815, 0.03124746520861403, 0.05902318439853504, 0.03756308574353105, 0.04600890373226976, 0.0690330924431201, 0.04639849241897629, 0.07358693105009391, 0.2798518426690238, -0.0037818138302955114, 0.30401703736788466, 0.2987773371566251, 0.05233971319770532, 0.04356940948397739, 0.08730394265765168, 0.015049435409669689, -0.0011879650078185492, 0.03199610949568849, 0.020293536322393346, 0.004764172016111142, 0.04531472292508923, 0.06867107500383968, 0.015430062276214398, 0.1134100534659356, -0.009039557853736613, 0.0018231786145566139, -0.02485228417185424, 0.35582734118677667, 0.10107579185601766, 0.10107508268381071, 0.387655131612059, 0.23400096388951913, 0.07324698997737274, 0.12410337250958965, 0.07527626239420507, 0.06815185541011753, 0.024462892680669146, 0.09842907728610124, 0.06793064029244208, 0.11620092339911516, 0.1868195539022079, 0.025292601

In [19]:
# this is using pandas 
%%time
from datetime import datetime
s=datetime.now()
def effect_prob():
    #creating a list of lists consisting of all the causes for all the effects
    names=[]
    for e in effect:
        # creating a list of causes for that particular effect
        rel_c1=[]
    
        df_e1 = df[df[e]>-1]
        # calculating the probability of the effect
        df_e1_prob = df_e1[e].sum()/len(df_e1[e])
        print(e,df_e1_prob)
        #checking for the cuases conditioned on effect that has probability greater than that of effect
        for i in df_cause.columns:
            df_e1_c1_prob=0
            df_e1_c1 = df_e1[df_e1[i]>-1]
            if(df_e1_c1[i].sum()!=0):
                #print(i,e)
                df_e1_c1_prob = (df_e1_c1[e].mul(df_e1_c1[i])).sum()/df_e1_c1[i].sum()
            if(df_e1_c1_prob>df_e1_prob):
                rel_c1.append(i)
        #print(rel_c1)
        diff=[0 for k in rel_c1]
        for i in rel_c1:
            ind=rel_c1.index(i)
            for j in rel_c1:
                if(i is not j):
                    # calculating the difference between probability of cause c1 with cause c2 with effect e and not cause c1 with cause c2 with effect e 
                    df_e1_c1 = df_e1[df_e1[i]>-1]
                    df_e1_c1_c2 = df_e1_c1[df_e1_c1[j]>-1]
                    if(((df_e1_c1_c2[i].mul(df_e1_c1_c2[j])).sum())!=0 and ((1-df_e1_c1_c2[i]).mul(df_e1_c1_c2[j])).sum()!=0):
                        df_e1_c1_c2_prob = (df_e1_c1_c2[i].mul(df_e1_c1_c2[j]).mul(df_e1_c1_c2[e])).sum()/(df_e1_c1_c2[i].mul(df_e1_c1_c2[j])).sum()

                        df_e1_notc1_c2_prob = ((1-df_e1_c1_c2[i]).mul(df_e1_c1_c2[j]).mul(df_e1_c1_c2[e])).sum()/((1-df_e1_c1_c2[i]).mul(df_e1_c1_c2[j])).sum()

                        e_diff=df_e1_c1_c2_prob-df_e1_notc1_c2_prob

                        diff[ind]=diff[ind]+e_diff
            diff[rel_c1.index(i)] = diff[rel_c1.index(i)]/(len(rel_c1)-1) 
        print(diff)
        names.append(rel_c1)
    return names

effect_prob()
print(datetime.now()-s)
#print(rel_c1_d)

Though if you run the code pandas code usually outperforms the cudf due to data size to see difference between cudf and pandas we need to run it on huge data chunks, you can check the example provided in the first cell for results as you can see the size of np array is huge 